In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


In [47]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [48]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        #Defining 2 convolutional layers
        self.convlay1 = nn.Conv2d(3, 6, 5, 1)
        self.convlay2 = nn.Conv2d(6, 16, 5, 1)
        #Defining pooling layer
        self.pool = nn.MaxPool2d(2,2)
        #Defining 3 fully connected layers
        self.fullyconlay1 = nn.Linear(16*29*29, 100)
        self.fullyconlay2 = nn.Linear(100, 50)
        self.fullyconlay3 = nn.Linear(50, 10) #Final output channels need to be 10

    def forward(self, image_data):
        #First pass
        image_data = F.relu(self.convlay1(image_data)) #Featured maps
        image_data = self.pool(image_data) #Pooled featured maps
        #Second pass
        image_data = F.relu(self.convlay2(image_data)) #Featured maps
        image_data = self.pool(image_data) #Pooled featured maps
        #Flatten layer
        image_data = torch.flatten(image_data, 1)
        #Pass through fully connected layers using relu
        image_data = F.relu(self.fullyconlay1(image_data))
        image_data = F.relu(self.fullyconlay2(image_data))
        return self.fullyconlay3(image_data)

In [49]:
import kagglehub

# Download latest version
dataset_path = kagglehub.dataset_download("karimabdulnabi/fruit-classification10-class")
dataset_path = os.path.join(dataset_path, "MY_data")
# Setup path to data folder
train_dir = os.path.join(dataset_path, "train")
test_dir = os.path.join(dataset_path, "test")
predict_dir = os.path.join(dataset_path, "predict")

train_dir, test_dir


('/root/.cache/kagglehub/datasets/karimabdulnabi/fruit-classification10-class/versions/1/MY_data/train',
 '/root/.cache/kagglehub/datasets/karimabdulnabi/fruit-classification10-class/versions/1/MY_data/test')

In [50]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        y_pred = model(X)


        loss = loss_fn(y_pred, y)
        train_loss += loss.item()


        optimizer.zero_grad()


        loss.backward()


        optimizer.step()


        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):
    model.eval()

    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Check DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Data is sent to target decvice
            X, y = X.to(device), y.to(device)
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            #Determine  accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [51]:
from tqdm.auto import tqdm

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):

    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }


    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)


        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        # Ensure all data is moved to CPU and converted to float for storage
        results["train_loss"].append(train_loss.item() if isinstance(train_loss, torch.Tensor) else train_loss)
        results["train_acc"].append(train_acc.item() if isinstance(train_acc, torch.Tensor) else train_acc)
        results["test_loss"].append(test_loss.item() if isinstance(test_loss, torch.Tensor) else test_loss)
        results["test_acc"].append(test_acc.item() if isinstance(test_acc, torch.Tensor) else test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

In [52]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [53]:
train_data = datasets.ImageFolder(train_dir, transform=transform)
test_data = datasets.ImageFolder(test_dir, transform=transform)

train_data, train_data

(Dataset ImageFolder
     Number of datapoints: 2301
     Root location: /root/.cache/kagglehub/datasets/karimabdulnabi/fruit-classification10-class/versions/1/MY_data/train
     StandardTransform
 Transform: Compose(
                Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ),
 Dataset ImageFolder
     Number of datapoints: 2301
     Root location: /root/.cache/kagglehub/datasets/karimabdulnabi/fruit-classification10-class/versions/1/MY_data/train
     StandardTransform
 Transform: Compose(
                Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ))

In [54]:
import os
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

torch.manual_seed(42)
train_dataloader = DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                        num_workers=NUM_WORKERS)

test_dataloader = DataLoader(test_data,
                                    batch_size=BATCH_SIZE,
                                    shuffle=False,
                                    num_workers=NUM_WORKERS)

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7feaeb3b6020>,
 <torch.utils.data.dataloader.DataLoader at 0x7feaeb3b58d0>)

In [55]:
torch.manual_seed(42)
model = CNN().to(device)
model

NUM_EPOCHS = 50
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)


from timeit import default_timer as timer
start_time = timer()


results = train(model=model,train_dataloader=train_dataloader,test_dataloader=test_dataloader,optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS)

end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.9180 | train_acc: 0.2535 | test_loss: 1.7554 | test_acc: 0.3163
Epoch: 2 | train_loss: 1.5040 | train_acc: 0.4338 | test_loss: 1.7411 | test_acc: 0.4100
Epoch: 3 | train_loss: 1.3376 | train_acc: 0.5020 | test_loss: 1.6799 | test_acc: 0.3835
Epoch: 4 | train_loss: 1.1892 | train_acc: 0.5724 | test_loss: 1.6757 | test_acc: 0.3902
Epoch: 5 | train_loss: 0.9422 | train_acc: 0.6569 | test_loss: 1.6945 | test_acc: 0.4441
Epoch: 6 | train_loss: 0.6979 | train_acc: 0.7585 | test_loss: 2.0223 | test_acc: 0.4138
Epoch: 7 | train_loss: 0.4575 | train_acc: 0.8512 | test_loss: 2.2666 | test_acc: 0.4489
Epoch: 8 | train_loss: 0.2795 | train_acc: 0.9169 | test_loss: 2.5465 | test_acc: 0.4583
Epoch: 9 | train_loss: 0.1384 | train_acc: 0.9627 | test_loss: 2.9342 | test_acc: 0.4432


KeyboardInterrupt: 